In [35]:
import pandas as pd
import bs4
import re

from tqdm.notebook import tqdm
tqdm.pandas()

### Getting the files
We have the raw files from the database.
These are extracted in **Microsoft SQL Manager Studio 18**, by calling a `SELECT * FROM TABLE`.
Then, on the result-set right-click and press `save result as`.

In [36]:
PATH_REQUESTS = 'data/database/Requests.csv'

df = pd.read_csv(PATH_REQUESTS,
                 encoding='UTF-8',
                 delimiter=';',
                 quotechar='"',
                 dtype=str,
                 usecols=['id', 'receivedDate', 'solutionDate'])

df = df.fillna('')

In [37]:
PATH_RELATION_HISTORY = 'data/database/RelationHistory.csv'

df_relation_history = pd.read_csv(PATH_RELATION_HISTORY,
                 encoding='UTF-8',
                 delimiter=';',
                 quotechar='"',
                 dtype=str,
                 usecols=['leftID', 'rightID', 'leftType', 'rightType', 'tblTimeStamp'])

df = df.fillna('')

In [38]:
df_relation_history = df_relation_history.sort_values(by='tblTimeStamp')
df_relation_history = df_relation_history[df_relation_history['leftType'].isin(['RequestService', 'RequestIncident'])]
df_relation_history = df_relation_history.drop_duplicates(subset=['rightID'], keep='last')

df = pd.merge(df, df_relation_history, left_on='id', right_on='leftID', how='left')
df = df.fillna('')

In [39]:
import numpy as np

df_with_communication = df[df['rightType'] == 'CommunicationSimple']
df_with_communication = df_with_communication.drop_duplicates(subset=['leftID'], keep='last')

df_without_communication = df[~df['leftID'].isin(np.unique(df_with_communication['leftID'].to_numpy()))]
df_without_communication = df_without_communication.drop_duplicates(subset=['leftID'], keep='last')

df = pd.concat([df_with_communication, df_without_communication])

In [40]:
from datetime import datetime

def get_last_received_time(x):

    if x['receivedDate'] == '':
        return 0

    if str(x['receivedDate'])[0] == '0':
        return 0

    received = datetime.strptime(str(x['receivedDate'][:-4]), "%Y-%m-%d %H:%M:%S")
    return received.timestamp()

def get_last_communication_time(x):

    if x['receivedDate'] == '':
        return 0

    if str(x['receivedDate'])[0] == '0':
        return 0

    result_communication = 0

    if x['rightType'] == 'CommunicationSimple':
        received = datetime.strptime(str(x['receivedDate'][:-4]), "%Y-%m-%d %H:%M:%S")
        solution = datetime.strptime(str(x['tblTimeStamp'][:-4]), "%Y-%m-%d %H:%M:%S")
        result_communication = int(solution.timestamp()) - int(received.timestamp())
        if result_communication <= 0:
            result_communication = 0

    return result_communication

def get_last_communication_time(x):

    if x['receivedDate'] == '':
        return 0

    if str(x['receivedDate'])[0] == '0':
        return 0

    result_communication = 0

    if x['rightType'] == 'CommunicationSimple':
        received = datetime.strptime(str(x['receivedDate'][:-4]), "%Y-%m-%d %H:%M:%S")
        solution = datetime.strptime(str(x['tblTimeStamp'][:-4]), "%Y-%m-%d %H:%M:%S")
        result_communication = int(solution.timestamp()) - int(received.timestamp())
        if result_communication <= 0:
            result_communication = 0

    return result_communication

def get_solution_time(x):

    if x['receivedDate'] == '' or x['solutionDate'] == '':
        return 0

    if str(x['receivedDate'])[0] == '0' or str(x['solutionDate'])[0] == '0':
        return 0

    received = datetime.strptime(str(x['receivedDate'][:-4]), "%Y-%m-%d %H:%M:%S")
    solution = datetime.strptime(str(x['solutionDate'][:-4]), "%Y-%m-%d %H:%M:%S")
    result_solution = int(solution.timestamp()) - int(received.timestamp())
    if result_solution <= 0:
        result_solution = 0

    return result_solution

def get_derived_completed_time(x):
    if x['solution_time'] == 0:
        return x['last_communication_time']
    if x['last_communication_time'] == 0:
        return x['solution_time']
    return np.min([x['solution_time'], x['last_communication_time']])

def get_time_bins(x, max_val, bins):
    i = ((x.name + 1) * bins)
    return int(i / (max_val + 1))

def get_time_bins_timestamp(x):
    if x.time_bins_solution == 0.0:
        return 240.0 + x.received_time
    elif x.time_bins_solution == 1.0:
        return 6300.0 + x.received_time
    elif x.time_bins_solution == 2.0:
        return 227460.0 + x.received_time
    elif x.time_bins_solution == 3.0:
        return 877980.0 + x.received_time
    elif x.time_bins_solution == 4.0:
        return 2678400.0 + x.received_time

In [41]:
df['solution_time'] = df.progress_apply(lambda x: get_solution_time(x), axis=1)
df['last_communication_time'] = df.progress_apply(lambda x: get_last_communication_time(x), axis=1)
df['derived_completed_time'] = df.progress_apply(lambda x: get_derived_completed_time(x), axis=1)
df['received_time'] = df.progress_apply(lambda x: get_last_received_time(x), axis=1)

  0%|          | 0/295432 [00:00<?, ?it/s]

  0%|          | 0/295432 [00:00<?, ?it/s]

  0%|          | 0/295432 [00:00<?, ?it/s]

  0%|          | 0/295432 [00:00<?, ?it/s]

In [42]:
df = df[df['received_time'] > 0]

In [43]:
df = df.sort_values(by='derived_completed_time')
df = df.reset_index()
df['time_bins_derived'] = df.progress_apply(lambda x: get_time_bins(x, len(df), 5), axis=1)

  0%|          | 0/277006 [00:00<?, ?it/s]

In [44]:
df = df.sort_values(by='solution_time')
df = df.reset_index()
df['time_bins_solution'] = df.progress_apply(lambda x: get_time_bins(x, len(df), 5), axis=1)
df['time_bins_solution_timestamp'] = df.progress_apply(lambda x: get_time_bins_timestamp(x), axis=1)

  0%|          | 0/277006 [00:00<?, ?it/s]

  0%|          | 0/277006 [00:00<?, ?it/s]

In [45]:
df = df[[
    'id',
    'received_time',
    'solution_time',
    'last_communication_time',
    'derived_completed_time',
    'time_bins_derived',
    'time_bins_solution',
    'time_bins_solution_timestamp',
]]
df.to_csv('data/label_time_complete.csv', index=False)